In [17]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import requests as req
from rich import print as rp
import polars as pl
from random import choice
import logging
from timeit_decorator import timeit
from pandarallel import pandarallel

# Configure logging
logging.basicConfig(level=logging.INFO)
pandarallel.initialize(progress_bar=True, nb_workers=8)

INFO: Pandarallel will run on 8 workers.
INFO: Pandarallel will use standard multiprocessing data transfer (pipe) to transfer data between the main process and workers.


In [2]:
# @timeit(log_level=logging.INFO)
def download_image(
    post_id: str = "CoGzQVFLqDB",
    path_download: str = "../data/images/",
    proxy_servers: list = [
        "100.80.84.42:9091",
        "100.70.13.68:9091",
        "100.122.187.59:9091",
        "100.66.160.80:9091",
    ],
) -> None:
    url = f"https://www.instagram.com/p/{post_id}/media/?size=l"
    # Create folder if not exist
    if not os.path.exists(path_download):
        os.makedirs(path_download)
    # Skip if file exist

    path_file = f"{path_download}{post_id}.jpg"
    if not os.path.exists(path_file):
        # Define proxy
        proxy_servers = proxy_servers
        proxy_server = choice(proxy_servers)
        proxies = {
            "http": f"socks5://{proxy_server}",
            "https": f"socks5://{proxy_server}",
        }
        # Download image
        response = req.get(url, proxies=proxies)
        if response.status_code == 200:
            try:
                # Save image
                with open(path_file, "wb") as file:
                    file.write(response.content)
                rp(f"[green bold]Downloaded: {post_id}.[/green bold]")
            except Exception as e:
                rp(f"[red bold]Failed due to an error: {post_id}, {e}.[/red bold]")
        else:
            rp(f"[red bold]Failed: {post_id}.[/red bold]")
    else:
        rp(f"[yellow bold]Skipped: {post_id}.[/yellow bold]")


download_image()

Skipped: CoGzQVFLqDB.

In [15]:
import re

df = pl.read_csv("../data/raw/consolidated_press.csv", ignore_errors=True)
df = df.select(["URL"]).to_pandas()
pattern = r"(p|tv)\/([A-Z,a-z,0-9,_,-]+)\/"  # Starting with p or tv; followed by a string of characters, numbers, _, or -; ending with /
df["POST_ID"] = df["URL"].apply(
    lambda x: re.findall(pattern, x)[0][1] if re.findall(pattern, x) else "None"
)
# print(df["URL"][0])
df.head()

,URL,POST_ID
0,https://www.instagram.com/p/C9EZeNnRNLY/,C9EZeNnRNLY
1,https://www.instagram.com/p/C9Dr_tjAL6S/,C9Dr_tjAL6S
2,https://www.instagram.com/p/C9DPywVNJ2w/,C9DPywVNJ2w
3,https://www.instagram.com/p/C9DPSZmM_RO/,C9DPSZmM_RO
4,https://www.instagram.com/p/C9DAH0bSs0z/,C9DAH0bSs0z


In [19]:
df_sample = df.sample(100)
# df_sample["POST_ID"].apply(download_image)
df_sample.parallel_apply(lambda x: download_image(x["POST_ID"]), axis=1)

2413     None
1628     None
534      None
5593     None
3606     None
         ... 
5804     None
12798    None
3576     None
2086     None
1977     None
Length: 100, dtype: object